### Задание 1.

**Задание**: обучите три классификатора: 

1) на токенах с высокой частотой 

2) на токенах со средней частотой 

3) на токенах с низкой частотой


Сравните полученные результаты, оцените какие токены наиболее важные для классификации.


### Задание 2.

найти фичи с наибольшей значимостью, и вывести их


### Задание 3.

1) сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report)

2) подобрать оптимальный размер для hashing векторайзера 

3) убедиться что для сетки нет переобучения

In [13]:
import pandas as pd
import numpy as np
import os
from multiprocessing import Pool
from tqdm.notebook import tqdm
from nltk.tokenize import word_tokenize
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer


from my_tool import txt_preprocessing, classificator

In [4]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = 1
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = 0
df = positive.append(negative)

In [5]:
df.head(2)

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1


### Задание 1

In [7]:
%%time
N = len(df)
with Pool(8) as p:
    df['prep_txt'] = tqdm(p.imap(txt_preprocessing.ru_preprocessing, df['text']), total = N)

  0%|          | 0/226834 [00:00<?, ?it/s]

Wall time: 5min 34s


In [8]:
df.head(2)

,text,label,prep_txt
0,"@first_timee хоть я и школота, но поверь, у на...",1,школоть поверить самый общество профилировать ...
1,"Да, все-таки он немного похож на него. Но мой ...",1,весь таки немного похожий мальчик равный


In [9]:
x_train, x_test, y_train, y_test = train_test_split(df.prep_txt, df.label, shuffle=True)

In [10]:
corpus = [token for tweet in df.prep_txt for token in word_tokenize(tweet)]
freq_dict = Counter(corpus)
freq_dict_sorted= sorted(freq_dict.items(), key=lambda x: -x[1])

In [11]:
t = np.array([freq for word, freq in freq_dict_sorted])
print(f'Общее число токенов в корпусе: {t.sum()}')
max_ = np.array([freq for word, freq in freq_dict_sorted if freq > 1000])
print(f'Число токенов в корпусе с частотой выше 1000: {max_.sum()}')
middle_ = np.array([freq for word, freq in freq_dict_sorted if freq > 100 and freq <= 1000])
print(f'Число токенов в корпусе с частотой >100 <1000: {middle_.sum()}')
min_ = np.array([freq for word, freq in freq_dict_sorted if freq <= 100])
print(f'Число токенов в корпусе с частотой ниже 100: {min_.sum()}')

Общее число токенов в корпусе: 1450128
Число токенов в корпусе с частотой выше 1000: 499939
Число токенов в корпусе с частотой >100 <1000: 491955
Число токенов в корпусе с частотой ниже 100: 458234


In [12]:
vec_list = {'max':[30000, 1000], 'middle':[1001, 100], 'min':[101, 1], 'total':[30000, 1]}
vec = {}
for key, range_ in vec_list.items():
    vec[key] = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, max_df=range_[0], 
                               min_df=range_[1])
print(vec)

{'max': CountVectorizer(max_df=30000, min_df=1000,
                tokenizer=<function word_tokenize at 0x0000019F868A6940>), 'middle': CountVectorizer(max_df=1001, min_df=100,
                tokenizer=<function word_tokenize at 0x0000019F868A6940>), 'min': CountVectorizer(max_df=101,
                tokenizer=<function word_tokenize at 0x0000019F868A6940>), 'total': CountVectorizer(max_df=30000,
                tokenizer=<function word_tokenize at 0x0000019F868A6940>)}


In [40]:
pool = Pool(processes=4)
res = []
for key, vectorizer in vec.items(): 
    r = pool.apply_async(classificator.RFCls, 
                         (key, vectorizer, x_train, y_train, x_test, y_test))
    res.append((key,r))
pool.close()
already = []
result = []
while len(already) < 4:
    for key, r in res:
        if r.ready() and key not in already:
            already.append(key)
            result.append(r.get())
            print(key, 'is ready!')
pool.join()

max is ready!
middle is ready!
total is ready!
min is ready!


In [43]:
col = ['Вариант', 'precision', 'recall', 'f1_score']
result_df = pd.DataFrame(result, columns=col)
result_df

,Вариант,precision,recall,f1_score
0,max,0.793357,0.628191,0.701179
1,middle,0.713209,0.640431,0.674863
2,total,0.677529,0.776879,0.723811
3,min,0.533838,0.666435,0.592813


#### Выводы по заданию

Общее число токенов в корпусе составило 1450128, для проведения расчётов я поделил их на 3 равные части. 


Получилось, что примерно 1/3 всего корпуса токенов приходится на токены с частотами выше 1000 (499939шт.) Еще 1/3 лежит в диапазоне частот от 1000 до 100 (491955шт.). Остальные с частотами ниже 100.

Результаты показали, что лучшие показатели были у модели, обученной на высокочастотных токенах. Модель на низкочастотных токенах показала худший результат. 
А как будет видно в задания 2, то низкочастотные токены в какой-то мере ухудшили работу модели, обученную на всём корпусе. Модель "max+middle" модель обученная без низкочастотных токенов, показала лучший результат чем "total" - модель обученная на всем корпусе

Для организации параллельного обучения моделей пришлось функцию вынести в отдельный модуль classificator. Если функция располагалась в данном ноутбуке, распараллелить не удалось. (Объяснить причину не смог) 

### Задание 2

In [46]:
vect = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, max_df=30000, 
                               min_df=100)

In [47]:
key, precision_, recall_, f1_score_, rfc = classificator.RFCls_model('max+middle', vect, 
                                                x_train, y_train, x_test, y_test)

In [62]:
score = pd.DataFrame([{'Вариант':key, 'precision':precision_, 
                      'recall':recall_,'f1_score':f1_score_}])

In [64]:
result_df.append(score)

,Вариант,precision,recall,f1_score
0,max,0.793357,0.628191,0.701179
1,middle,0.713209,0.640431,0.674863
2,total,0.677529,0.776879,0.723811
3,min,0.533838,0.666435,0.592813
0,max+middle,0.736937,0.718832,0.727772


In [104]:
sorted(rfc.feature_importances_, reverse=True)[:10]

[0.10076841025162003,
 0.050276145990286195,
 0.010411144047479172,
 0.007436223162000596,
 0.006964873608564388,
 0.006319206111295495,
 0.005330647185699282,
 0.004490118993092208,
 0.004393730904472918,
 0.00409859834959355]

In [111]:
index_sort = np.argsort(rfc.feature_importances_)[::-1]
index_sort[:10]

array([ 230, 1298,  100, 1451, 1244,   51,   25,  416,   58,  325],
      dtype=int64)

In [ ]:
feature = vect.get_feature_names()

In [112]:
for index in index_sort[:10]:
    print(feature[index])

грустный
счастливый
весь
хотеть
спасибо
блин
ахах
знать
болеть
жаль


#### Выводы по заданию

грустный, счастливый


это токены вставленные при замене смайлов. Можно сделать вывод что подобные конструкции несут большую информационную нагрузку. 
Необходимо при работе с текстом  уделять им больше внимания.

### Задание 3

In [14]:
vect_CV = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, max_df=30000, 
                               min_df=100)

In [27]:
vect_CV.fit_transform(x_train)
print(len(vect_CV.get_feature_names()))

1525


In [15]:
vect_TF = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, max_df=30000, 
                               min_df=100)

In [26]:
vect_HS_max = HashingVectorizer(ngram_range=(1, 1), n_features=1525, tokenizer=word_tokenize)

In [29]:
vect_HS_min = HashingVectorizer(ngram_range=(1, 1), n_features=500, tokenizer=word_tokenize)

In [30]:
vect_HS_middle = HashingVectorizer(ngram_range=(1, 1), n_features=1000, tokenizer=word_tokenize)

In [34]:
list_vec = {'CountVectorizer': vect_CV, 'TfidfVectorizer': vect_TF, 
            'HashingVectorizer_max': vect_HS_max, 
            'HashingVectorizer_middle': vect_HS_middle, 
            'HashingVectorizer_min': vect_HS_min}

In [37]:
pool = Pool(processes=5)
res = []
for key, vectorizer in list_vec.items(): 
    r = pool.apply_async(classificator.RFCls_model, 
                         (key, vectorizer, x_train, y_train, x_test, y_test))
    res.append((key,r))
pool.close()
already = []
result = []
while len(already) < 5:
    for key, r in res:
        if r.ready() and key not in already:
            already.append(key)
            result.append(r.get())
            print(key, 'is ready!')
pool.join()

CountVectorizer is ready!
TfidfVectorizer is ready!
HashingVectorizer_max is ready!
HashingVectorizer_middle is ready!
HashingVectorizer_min is ready!


In [39]:
col = ['Вариант', 'precision', 'recall', 'f1_score', 'модель']
result_df = pd.DataFrame(result, columns=col)
result_df

,Вариант,precision,recall,f1_score,модель
0,CountVectorizer,0.742462,0.720003,0.731060,"(DecisionTreeClassifier(max_features='auto', r..."
1,TfidfVectorizer,0.770529,0.724216,0.746655,"(DecisionTreeClassifier(max_features='auto', r..."
2,HashingVectorizer_max,0.737078,0.731513,0.734285,"(DecisionTreeClassifier(max_features='auto', r..."
3,HashingVectorizer_middle,0.732597,0.723375,0.727957,"(DecisionTreeClassifier(max_features='auto', r..."
4,HashingVectorizer_min,0.730791,0.700636,0.715396,"(DecisionTreeClassifier(max_features='auto', r..."


In [49]:
pred = {}
i = 0
for key, vectorizer in list_vec.items(): 
    vectorizer.fit_transform(x_train)
    pred[key] = result_df.iloc[i]['модель'].predict(vectorizer.transform(x_test))
    i +=1
    
for key, predict in pred.items(): 
    print(f'classification_report для {key}:\n')
    print(classification_report(predict, y_test))
    print()
    print()

C:\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


classification_report для CountVectorizer:

              precision    recall  f1-score   support

           0       0.70      0.73      0.71     27023
           1       0.74      0.72      0.73     29686

    accuracy                           0.72     56709
   macro avg       0.72      0.72      0.72     56709
weighted avg       0.72      0.72      0.72     56709



classification_report для TfidfVectorizer:

              precision    recall  f1-score   support

           0       0.70      0.75      0.72     26080
           1       0.77      0.72      0.75     30629

    accuracy                           0.73     56709
   macro avg       0.73      0.74      0.73     56709
weighted avg       0.74      0.73      0.73     56709



classification_report для HashingVectorizer_max:

              precision    recall  f1-score   support

           0       0.72      0.73      0.72     27702
           1       0.74      0.73      0.73     29007

    accuracy                           0

In [638]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import Conv1D, GRU, Dropout, Concatenate, BatchNormalization
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [639]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
valid_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [640]:
train_data = train_data.batch(64)
valid_data = valid_data.batch(64)

In [641]:
AUTOTUNE = tf.data.AUTOTUNE
train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [642]:
class myNet(tf.keras.Model):
    def __init__(self, embedding_dim = 200, vocab_size = 10000, seq_len = 50):
        super(myNet, self).__init__()
        self.vectorize_layer = TextVectorization(max_tokens=vocab_size, output_mode='int',
                              output_sequence_length=seq_len)
        text_data = train_data.map(lambda x, y: x)
        self.vectorize_layer.adapt(text_data)        
        
        self.emb = Embedding(vocab_size, embedding_dim, name="embedding")
        self.bn1 = BatchNormalization()
        self.gru1 = GRU(embedding_dim, return_sequences=True, return_state=True, dropout=0.15)
        self.gru2 = GRU(embedding_dim, dropout=0.15)
        self.bn2 = BatchNormalization()        
        self.fc0 = Dense(embedding_dim*4, activation='relu')
        self.dp = Dropout(0.15)
        self.fc1 = Dense(embedding_dim, activation='relu')
        self.fc2 = Dense(1)


    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        x = self.bn1(x) 
        x, _ = self.gru1(x) 
        x = self.gru2(x)
        x = self.fc0(x)
        x = self.bn2(x)         
        x = self.dp(x)         
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [643]:
mmodel = myNet(embedding_dim = 200)

In [644]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
mmodel.compile(optimizer=opt,
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [645]:
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
2659/2659 [==============================] - 976s 365ms/step - loss: 0.5631 - accuracy: 0.6450 - val_loss: 0.6155 - val_accuracy: 0.6737
Epoch 2/5
2659/2659 [==============================] - 996s 375ms/step - loss: 0.5221 - accuracy: 0.6922 - val_loss: 0.5410 - val_accuracy: 0.6621
Epoch 3/5
2659/2659 [==============================] - 979s 368ms/step - loss: 0.5122 - accuracy: 0.7070 - val_loss: 0.5218 - val_accuracy: 0.7207
Epoch 4/5
2659/2659 [==============================] - 981s 369ms/step - loss: 0.5095 - accuracy: 0.7033 - val_loss: 0.5159 - val_accuracy: 0.6796
Epoch 5/5
2659/2659 [==============================] - 980s 369ms/step - loss: 0.5071 - accuracy: 0.7118 - val_loss: 0.5183 - val_accuracy: 0.6754


#### Выводы по заданию

На данном корпусе лучший результат оказался у TfidfVectorizer, что вполне ожидаемо.

Для нахождения оптимального размера для hashing векторайзера не хватает вводных данных. Не известно по какому параметру оптимизировать по метрикам или скорости исполнения. Чем быстрее работает модель, тем меньше размер, но при этом падают метрики. 

В расчётах HashingVectorizer в случае максимум я взял размер равный длине словаря в  CountVectorizer (vect_CV.get_feature_names) при таком объёме метрики совпали. 

Нейронная сеть показала худший результат в данном случае. Как показывает практика, нейронная сеть не всегда является лучшим решением задачи.  
